In [1]:
pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split, explode, count
import re

In [3]:
class SparkWordCount:

    def __init__(self, input_path, output_path):
        self.input_path = input_path
        self.output_path = output_path
        self.spark = SparkSession.builder \
            .appName("Spark Word Count") \
            .getOrCreate()

    def word_count(self):
        # Read the input data as text file(s) into a DataFrame
        lines = self.spark.read.text(self.input_path)

        # Define the word count logic
        def count_words(line):
            regex = re.compile(r"\b\w+\b")
            words = regex.findall(line.lower())
            return [(word, 1) for word in words]

        # Apply the word count logic and explode the result to get (word, count) pairs
        word_counts = lines.rdd.flatMap(count_words).reduceByKey(lambda a, b: a + b)

        # Convert the RDD back to DataFrame and add column names
        word_counts_df = word_counts.toDF(["word", "count"])

        # Save the word counts to the specified output path in CSV format
        word_counts_df.write.csv(self.output_path, header=True, mode="overwrite")

        # Stop the Spark session
        self.spark.stop()
        
if __name__ == '__main__':
    # Replace 'input_path' and 'output_path' with your desired file paths
    input_path = "input.txt"
    output_path = "output.txt"
    word_counter = SparkWordCount(input_path, output_path)
    word_counter.word_count()

RuntimeError: Java gateway process exited before sending its port number